In [22]:
pwd


'C:\\Users\\mwolf\\3D Objects\\python veri bilimi\\SPARK\\spark calisma'

In [23]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv('adult.data') 

In [24]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("adult.test") \

# Veri Birleştirme

In [25]:
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(5).toPandas().head()

age          workclass    fnlwgt   education  education_num  \
0   39          State-gov   77516.0   Bachelors           13.0   
1   50   Self-emp-not-inc   83311.0   Bachelors           13.0   
2   38            Private  215646.0     HS-grad            9.0   
3   53            Private  234721.0        11th            7.0   
4   28            Private  338409.0   Bachelors           13.0   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  output  
0        2174.0           0.0            40.0   United-States   <=50K  
1           0.0           0.0            13.0   United-States   <=50K  
2           0.0           0.0            40.0   United-States   <=50K  
3           0.0           0.0            40.0   United-States   <=50K  
4           0.0           0.0            40.0            Cuba   <=50K

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER //////////////////////////////////////////////// /*

  1. Tüm sütunları boşluk kontrolü yap.
  2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
      ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
      ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
      Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
  3. workclass niteliğinde never-worked ve without-pay sınıfları ve
      occupation niteliğinde  Armed-Forces  sınıfı
    çok az tekrarlanmış. Veri setinden çıkarılabilir.
  4. education niteliğindeki:
          1st-4th, 5th-6th, 7th-8th: elementary-school
          9th, 10th, 11th, 12th: high-school
          Masters, Doctorate: high-education
          Bachelors, Some-college: undergraduate
     sınıfları yukarıdaki gibi birleştirilebilir.
  5. native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
  6. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı
 */

# ==== VERİ TEMİZLİĞİ BAŞLIYOR ===========

## 1. Tüm sütunları boşluk kontrolü yap.

In [26]:
from pyspark.sql.functions import *

In [27]:
adult_whole_df1 =  adult_whole_df \
.withColumn("workclass",trim(col("workclass")))

In [28]:
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", trim(col("workclass"))) \
.withColumn("education", trim(col("education"))) \
.withColumn("marital_status", trim(col("marital_status"))) \
.withColumn("occupation", trim(col("occupation"))) \
.withColumn("relationship", trim(col("relationship"))) \
.withColumn("race", trim(col("race"))) \
.withColumn("sex", trim(col("sex"))) \
.withColumn("native_country", trim(col("native_country"))) \
.withColumn("output", trim(col("output"))) \

In [29]:
adult_whole_df.count()

48842

In [30]:
adult_whole_df1.count()

48842

# 2. OUTPUT İÇİNDEKİ "." TEMİZLİĞİ

Her ne kadar taslak planda output temizliğini son madde olarak yazmış olsak da. İşin kolay olması ve diğer temizlik işlemlerini etkiliyor olması nedeniyle. Bu temizliği en başta yapalım.

    // . içerenleri replace yapalım.

In [31]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
.withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

In [32]:
# Temizlik sonucunu görelim.
adult_whole_df2.groupBy(col("output")).agg({"*":"count"}) \
.toPandas().head()

output  count(1)
0  <=50K     37155
1   >50K     11687

# 3. NULL KONTROLÜ

// Her bir sütun için for dongüsü ile o sütunda filter() metodu ile null kontrolü yapalım ve bunları sayalım. Eğer o sütunda en az bir adet null varsa yani >0 ise o sütunda null olduğunu ekrana yazdıralım.

In [33]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [34]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count() > 0 ):
        print(sayac_for_null, ". ", sutun, " içinde null var.")
    else:
        print(sayac_for_null, sutun)
    sayac_for_null += 1

1 age
2 workclass
3 fnlwgt
4 education
5 education_num
6 marital_status
7 occupation
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 native_country
15 output


// Herhangi bir null yok.

# 4. "?" KONTROLLERİ

In [35]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count() > 0 ):
        print(sayac_for_question, ". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, sutun)
    sayac_for_question += 1

1 age
2 .  workclass  içinde ? var.
3 fnlwgt
4 education
5 education_num
6 marital_status
7 .  occupation  içinde ? var.
8 relationship
9 race
10 sex
11 capital_gain
12 capital_loss
13 hours_per_week
14 .  native_country  içinde ? var.
15 output


// workclass, occupation, native_country içinde ? var.

// Bunların hedef değişken output ile ilişkisini inceleyelim.

In [36]:
adult_whole_df2.select("workclass","occupation","native_country","output") \
.filter(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
    
) \
.groupBy("workclass","occupation","native_country","output").count() \
.orderBy(col("count").desc()) \
.toPandas().head(50)

workclass         occupation      native_country output  count
0                  ?                  ?       United-States  <=50K   2284
1                  ?                  ?       United-States   >50K    246
2            Private      Other-service                   ?  <=50K    100
3            Private              Sales                   ?  <=50K     55
4            Private     Prof-specialty                   ?  <=50K     51
5            Private       Craft-repair                   ?  <=50K     48
6            Private     Prof-specialty                   ?   >50K     48
7                  ?                  ?              Mexico  <=50K     48
8            Private       Adm-clerical                   ?  <=50K     47
9            Private  Machine-op-inspct                   ?  <=50K     42
10                 ?                  ?                   ?  <=50K     40
11           Private    Exec-managerial                   ?  <=50K     37
12           Private   Transport-moving                   ?  <=50K     28
13           Private    Exec-managerial                   ?   >50K     24
14           Private  Handlers-cleaners                   ?  <=50K     21
15           Private       Craft-repair                   ?   >50K     18
16                 ?                  ?              Canada  <=50K     16
17           Private       Tech-support                   ?  <=50K     14
18  Self-emp-not-inc              Sales                   ?  <=50K     13
19                 ?                  ?             Germany  <=50K     13
20                 ?                  ?               South  <=50K     12
21         Local-gov     Prof-specialty                   ?  <=50K     11
22                 ?                  ?         Philippines  <=50K     11
23         State-gov     Prof-specialty                   ?  <=50K     11
24           Private    Priv-house-serv                   ?  <=50K     10
25      Never-worked                  ?       United-States  <=50K     10
26           Private       Adm-clerical                   ?   >50K     10
27           Private              Sales                   ?   >50K     10
28       Federal-gov     Prof-specialty                   ?   >50K     10
29  Self-emp-not-inc    Exec-managerial                   ?  <=50K      9
30                 ?                  ?         Puerto-Rico  <=50K      9
31      Self-emp-inc    Exec-managerial                   ?   >50K      9
32  Self-emp-not-inc       Craft-repair                   ?  <=50K      9
33                 ?                  ?               China  <=50K      9
34                 ?                  ?              Taiwan  <=50K      9
35           Private       Tech-support                   ?   >50K      8
36      Self-emp-inc    Exec-managerial                   ?  <=50K      8
37                 ?                  ?         El-Salvador  <=50K      8
38                 ?                  ?             England  <=50K      8
39      Self-emp-inc              Sales                   ?  <=50K      7
40  Self-emp-not-inc     Prof-specialty                   ?   >50K      7
41         State-gov     Prof-specialty                   ?   >50K      7
42      Self-emp-inc              Sales                   ?   >50K      6
43                 ?                  ?  Dominican-Republic  <=50K      6
44                 ?                  ?               Haiti  <=50K      6
45         Local-gov     Prof-specialty                   ?   >50K      6
46           Private    Farming-fishing                   ?  <=50K      6
47  Self-emp-not-inc     Prof-specialty                   ?  <=50K      6
48                 ?                  ?                   ?   >50K      6
49           Private      Other-service                   ?   >50K      6

// Soru işaretlerinin dağılımı ve hedef değişken ile ilgisi tesadüfi görünüyor. Bu durumda ? işareti içeren satırları veri setinden çıkaralım.

In [37]:
adult_whole_df3 = adult_whole_df2 \
.filter(~(
    col("workclass").contains("?") |
    col("occupation").contains("?") |
    col("native_country").contains("?")
))

print(adult_whole_df2.count())
print(adult_whole_df3.count())

48842
45222


# 5. ZAYIF SINIFLARIN KALDIRILMASI

// workclass niteliğinde never-worked ve without-pay sınıfları ve
occupation niteliğinde  Armed-Forces  sınıfı
  çok az tekrarlanmış. Veri setinden çıkarılabilir.

In [38]:
adult_whole_df4 = adult_whole_df3 \
.filter(~(
    col("workclass").contains("never-worked") | col("workclass").contains("without-pay") | 
       col("occupation").contains("Armed-Forces") | col("native_country").contains("Holand-Netherlands")
))

print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45207


# 6. EĞİTİM DURUMUYLA İLGİLİ KATEGORİLERİ BİRLEŞTİRME

    // education niteliğindeki:
      1st-4th, 5th-6th, 7th-8th: elementary-school
    9th, 10th, 11th, 12th: high-school
    Masters, Doctorate: high-education
    Bachelors, Some-college: undergraduate
    sınıfları yukarıdaki gibi birleştirilebilir.

In [39]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged",
when (col("education").isin("1st-4th", "5th-6th", "7th-8th"),"Elementary-School"))

In [40]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged",
when(col("education").isin("1st-4th", "5th-6th", "7th-8th"), "Elementary-School") 
.when(col("education").isin("9th", "10th", "11th", "12th"), "High-School") 
.when(col("education").isin("Masters","Doctorate"), "Postgraduate")
.when(col("education").isin("Bachelors","Some-college"), "Undergraduate")
.otherwise(col("education"))
                                            
)

adult_whole_df5.select("education","education_merged").toPandas().head(20)

education   education_merged
0      Bachelors      Undergraduate
1      Bachelors      Undergraduate
2        HS-grad            HS-grad
3           11th        High-School
4      Bachelors      Undergraduate
5        Masters       Postgraduate
6            9th        High-School
7        HS-grad            HS-grad
8        Masters       Postgraduate
9      Bachelors      Undergraduate
10  Some-college      Undergraduate
11     Bachelors      Undergraduate
12     Bachelors      Undergraduate
13    Assoc-acdm         Assoc-acdm
14       7th-8th  Elementary-School
15       HS-grad            HS-grad
16       HS-grad            HS-grad
17          11th        High-School
18       Masters       Postgraduate
19     Doctorate       Postgraduate

// Evet yeni bir sütun ekledik. education_merged. Daha sade.

In [30]:
adult_whole_df5.toPandas().head()

age         workclass    fnlwgt  education  education_num  \
0   39         State-gov   77516.0  Bachelors           13.0   
1   50  Self-emp-not-inc   83311.0  Bachelors           13.0   
2   38           Private  215646.0    HS-grad            9.0   
3   53           Private  234721.0       11th            7.0   
4   28           Private  338409.0  Bachelors           13.0   

       marital_status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital_gain  capital_loss  hours_per_week native_country output  \
0        2174.0           0.0            40.0  United-States  <=50K   
1           0.0           0.0            13.0  United-States  <=50K   
2           0.0           0.0            40.0  United-States  <=50K   
3           0.0           0.0            40.0  United-States  <=50K   
4           0.0           0.0            40.0           Cuba  <=50K   

  education_merged  
0    Undergraduate  
1    Undergraduate  
2          HS-grad  
3      High-School  
4    Undergraduate

// Sütun sırasını değiştirelim. education_merge'i education yanına alalım

In [31]:
nitelik_siralama = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race",
     "sex", "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]

adult_whole_df6 = adult_whole_df5.select(nitelik_siralama)
adult_whole_df6.toPandas().head()

workclass  education education_merged      marital_status  \
0         State-gov  Bachelors    Undergraduate       Never-married   
1  Self-emp-not-inc  Bachelors    Undergraduate  Married-civ-spouse   
2           Private    HS-grad          HS-grad            Divorced   
3           Private       11th      High-School  Married-civ-spouse   
4           Private  Bachelors    Undergraduate  Married-civ-spouse   

          occupation   relationship   race     sex native_country  age  \
0       Adm-clerical  Not-in-family  White    Male  United-States   39   
1    Exec-managerial        Husband  White    Male  United-States   50   
2  Handlers-cleaners  Not-in-family  White    Male  United-States   38   
3  Handlers-cleaners        Husband  Black    Male  United-States   53   
4     Prof-specialty           Wife  Black  Female           Cuba   28   

     fnlwgt  education_num  capital_gain  capital_loss  hours_per_week output  
0   77516.0           13.0        2174.0           0.0            40.0  <=50K  
1   83311.0           13.0           0.0           0.0            13.0  <=50K  
2  215646.0            9.0           0.0           0.0            40.0  <=50K  
3  234721.0            7.0           0.0           0.0            40.0  <=50K  
4  338409.0           13.0           0.0           0.0            40.0  <=50K

In [32]:
adult_whole_df6 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep",",") \
.option("header","True") \
.csv("D:\\Datasets\\adult_preprocessed_pyspark")

In [21]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataCleaning") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()